In [15]:
from pyannote.audio import Pipeline
import os
from dotenv import load_dotenv

load_dotenv()

huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")

pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=huggingface_token)

In [8]:
import torch
# pipeline.to(torch.device("cuda"))

# apply pretrained pipeline
diarization = pipeline("../training_database/wavs/file1.wav")

# print the result
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")
# start=0.2s stop=1.5s speaker_0
# start=1.8s stop=3.9s speaker_1
# start=4.2s stop=5.7s speaker_0
# ...

C:\Users\verho\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


start=4.9s stop=5.5s speaker_SPEAKER_00
start=5.8s stop=8.4s speaker_SPEAKER_00
start=8.7s stop=10.7s speaker_SPEAKER_00
start=11.5s stop=13.4s speaker_SPEAKER_00
start=14.0s stop=17.0s speaker_SPEAKER_00
start=17.3s stop=17.6s speaker_SPEAKER_01
start=18.1s stop=18.9s speaker_SPEAKER_01
start=20.9s stop=21.0s speaker_SPEAKER_00
start=21.0s stop=22.5s speaker_SPEAKER_01
start=23.2s stop=24.4s speaker_SPEAKER_01
start=25.4s stop=26.5s speaker_SPEAKER_01
start=27.4s stop=28.8s speaker_SPEAKER_01
start=30.9s stop=33.1s speaker_SPEAKER_01
start=33.8s stop=35.0s speaker_SPEAKER_01
start=35.0s stop=37.2s speaker_SPEAKER_00
start=37.3s stop=37.9s speaker_SPEAKER_00
start=39.7s stop=40.7s speaker_SPEAKER_00
start=41.0s stop=42.0s speaker_SPEAKER_00
start=42.0s stop=42.5s speaker_SPEAKER_01
start=43.7s stop=43.7s speaker_SPEAKER_00
start=43.7s stop=44.7s speaker_SPEAKER_01
start=45.3s stop=45.4s speaker_SPEAKER_00
start=45.4s stop=45.5s speaker_SPEAKER_01
start=45.5s stop=45.6s speaker_SPEAKER_

In [9]:
print(len(diarization))

135


In [10]:
tracks = []
for segment in diarization.itersegments():
    print(type(segment))
    track = diarization.get_tracks(segment)
    tracks.append(track)
    print(type(next(iter(track))))

<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segment.Segment'>
<class 'str'>
<class 'pyannote.core.segmen

In [14]:
from pydub import AudioSegment

audio = AudioSegment.from_wav("../training_database/wavs/file1.wav")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    segment = audio[turn.start * 1000:turn.end * 1000]  # Convert seconds to milliseconds
    segment.export(f"./output/pyannote/speaker_{speaker}_{turn.start:.1f}-{turn.end:.1f}.wav", format="wav")
